# Using pandasGEXpress with .gct/x files 

--------------------------------------------------------------------------------
## A sample tutorial using data from [GEO GSE70138](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE70138)

Author: Oana Enache

Last updated February 23rd, 2018

For questions and issue reports, please file an issue at github.com/cmap/cmapPy. 

--------------------------------------------------------------------------------

## Table of Contents

We have developed this tutorial to loosely follow a workflow we recommend for .gct/x files downloaded from GEO GSE70138. 

[Background info](#background)

[Obtaining data from GEO](#getdata)

[Activate your conda environment](#conda)

[Parse in metadata only](#metadataparse)

[Parsing in a gct/x file](#parsegctx)

[Add/delete annotations](#annotations)

[Slicing gct/x files and GCToo instances](#slicing)

[Merging gct/x files](#merging)

[Construct a GCToo instance](#newGCToo)

[Using GCToo instances with pandas tools](#pandas)

[Plotting with GCToo objects](#plotting)

[Write a GCToo instance to file](#writing)

[Converting .gct <-> .gctx](#converting)

--------------------------------------------------------------------------------

<a id="background"></a>


## Background: What's .gct, .gctx, and GCToo? 

For more details on each of these topics, please visit clue.io/code. 


### The .gct format 

![Sample GCT file](GCT_mockup.png)

The GCT file format, a tab-delimited text-based format pairing matrix expression values with row and column metadata, allowing comparison of both transcriptional and contextual differences across samples. A schematic of a sample GCT file is pictured above. 

### The .gctx format

![Sample GCTX file](GCTX_mockup.png)

The GCTX file format was developed to improve serialization performance of GCT, particularly for large files. GCTX is an HDF5-based data format that can contain the same content as one or more GCT files in a hierarchically-structured, compressed, and easily accessible form (full file specification available at clue.io/code). Amongst other features, the HDF5 format allows users the ability to parse a subset of a large data set without reading the entire data set into memory first (using hyperslab selection). A schematic of a sample GCTX file is pictured above.

### The GCToo object 

`GCToo` objects are instances of the GCToo class we developed to represent .gct and .gctx files in a form compatible with Python's [PANDAS](http://pandas.pydata.org/) library. In essence, GCToo is a container of 3 pandas DataFrames (optionally, can also include a multi-index DataFrame of these 3 DataFrames combined) as well as two attributes representing the file source and version respectively. For more information, please see [our documentation](https://clue.io/cmapPy/pandasGEXpress.html#gctoo-class). 

<a id="getdata"></a>
## Obtaining Data from Gene Expression Omnibus (GEO)

[Gene Expression Omnibus](https://www.ncbi.nlm.nih.gov/geo/), or GEO, is a NCBI-maintained public repository of array and sequence genomics data. As of April 2017, we have provided 1.7 million profiles to GEO at the following accessions:

* [GSE70138](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE70138)

* [GSE92742](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE92742)

* [GSE92743](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE92743)

Please note that this does not represent the only data available in .gct and .gctx format; we simply mention it as a public resource for freely available data in these formats, and will use it as an example in this tutorial. 

Data can be downloaded from GEO using either using FTP or HTTP; see accession pages above for details. Details on what content is within each file published is clarified in the [GEO User Guide](https://docs.google.com/document/d/1q2gciWRhVCAAnlvF2iRLuJ7whrGP6QjpsCMq1yWz7dU/edit?usp=sharing). 

<a id="conda"></a>
## Activate your cmapPy conda environment 

If you haven't done so already, [create a conda environment to use for your analyses](https://clue.io/cmapPy/build.html#install).

For this example, we've named our conda environment ``my_cmapPy_env``. 

**Be sure to activate your environment before proceeding!**

``source activate my_cmapPy_env``

<a id="metadataparse"></a>
## Parse in metadata only 

Although we are in the process of developing tools to provide out of core functionality for larger that memory .gctx files, for now some .gctx files may be larger than the RAM you have available on your computer (note, we do not recommend the use of .gct for files above ~1000 samples). If this is the case, you have two options:

    1. Obtain access to a server/instance that has more RAM. 
    2. Read the metadata in first, find the sample (column) id values ("cids") that you're actually interested in, and only parse those cids in. 

### How to parse in metadata only from GEO

This would be applicable if you were interested in only certain samples (or certain compound names, cell types, etc). Here, we parse in the metadata from the level 5 (replicate-collapsed signature) data on GSE70138.

#### Sample (column) annotations 

On GEO, because of submission requirements annotations file are stored as separate tab-delimited text files. For level 5 data, we're interested in the "sig" related data tables. For this example, we previously downloaded and unzipped "GSE70138_Broad_LINCS_sig_info_2017-03-06.txt.gz" (corresponding to identifying information about the samples; if you wanted metrics-related information for filtering, explore the equivalently labelled "sig metrics" file). 

Then, we read this in to a pandas DataFrame: 

In [32]:
import pandas as pd

sig_info = pd.read_csv("GSE70138_Broad_LINCS_sig_info_2017-03-06.txt", sep="\t")

In [33]:
# look at the metadata fields provided 
sig_info.columns

Index([u'sig_id', u'pert_id', u'pert_iname', u'pert_type', u'cell_id',
       u'pert_idose', u'pert_itime', u'distil_id'],
      dtype='object')

As an illustrative example, say we're only interested in looking at expression data corresponding to cells treated as vorinostat. So, we first want to find the sig_id values that correspond to vorinostat-treated samples:

In [34]:
# get all samples (across all cell types, doses, and other treatment conditions) that were treated with vorinostat
vorinostat_ids = sig_info["sig_id"][sig_info["pert_iname"] == "vorinostat"]
# how many samples are there in this data set?
print("number of samples treated with vorinostat:")
len(vorinostat_ids)

number of samples treated with vorinostat:


210

#### Gene (row) annotations 

Analogously, you might be interested in only parsing in a specific row space; for example, you might be interested at only only examining patterns in expression of [landmark genes](https://clue.io/connectopedia/what_are_landmark_genes). The metadata column identifying landmark membership is `pr_is_lm` (for more details on other metadata header meanings, please see the [GEO User Guide](https://docs.google.com/document/d/1q2gciWRhVCAAnlvF2iRLuJ7whrGP6QjpsCMq1yWz7dU/edit#heading=h.l6bq0r1aih50)). 

So, we would parse in the gene annotations and obtain a list of only the row identifiers corresponding to landmark genes. **Note that for Entrez ids (as below), you need to specify that the data type annotations are strings (dtype=str argument below); otherwise, the parser currently behaves as if you're subsetting by index.** 

In [35]:
gene_info = pd.read_csv("GSE70138_Broad_LINCS_gene_info_2017-03-06.txt", sep="\t", dtype=str)
gene_info.columns

Index([u'pr_gene_id', u'pr_gene_symbol', u'pr_gene_title', u'pr_is_lm',
       u'pr_is_bing'],
      dtype='object')

In [36]:
# again, note that because gene_info dtypes are str, you need to find ids equal to "1" not 1 
landmark_gene_row_ids = gene_info["pr_gene_id"][gene_info["pr_is_lm"] == "1"]

Once you have the row and/or column identifiers you're interested in, parse only these ids from the GCTX file (see [Parse in a gctx](#parsegctx) below). 

### Side note: How to parse in metadata only from a metadata-annotated gctx

Note. *This is not specifically relevant to the data available on GEO*, but if you have a .gctx file with row/column annotations and want to only look at the metadata annotations, you can also read only these fields in from the gctx. You would do this as follows:

``from cmapPy.pandasGEXpress import parse`` 

``gctx_metadata_only = parse("path/to/gctx.gctx", metadata_only=True)``

In [37]:
from cmapPy.pandasGEXpress import parse

my_col_metadata = parse("GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328_2017-03-06.gctx", col_meta_only=True)
my_col_metadata[0:5]

chd
cid
REP.A001_A375_24H:A03
REP.A001_A375_24H:A04
REP.A001_A375_24H:A05
REP.A001_A375_24H:A06
REP.A001_A375_24H:A07


<a id="parsegctx"></a>
# Parse in a gctx

Now we're ready to read in expression data! For parsing, here are your options:

1. Read in an entire gctx file
2. Read in a subset of ids 

Generally, we recommend choice 2. 

## Example 1: Parse in a subset of samples/columns 

We'll demonstrate how to do so here using the ``vorinostat_ids`` we obtained [in the previous section](#metadataparse). 

In [38]:
from cmapPy.pandasGEXpress import parse
vorinostat_only_gctoo = parse("GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328_2017-03-06.gctx", cid=vorinostat_ids)

In [39]:
# Check to make sure we have all 210 colums of expression data: 
vorinostat_only_gctoo.data_df.shape #yup! 

(12328, 210)

## Example 2: Parse in a subset of genes/rows

We'll demonstrate how to do so here using the ``landmark_gene_row_ids`` we obtained [in the previous section](#metadataparse). 

In [40]:
landmark_only_gctoo = parse("GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328_2017-03-06.gctx", rid = landmark_gene_row_ids)

In [41]:
# Check to make sure we only have 978 rows of expression data: 
landmark_only_gctoo.data_df.shape #yup! 

(978, 118050)

Notes:

* If you did want to read in the entire file (option 1 above), you would just parse in the gct/x without the ``cid`` argument.
* Subsetting as demonstrated above is also available for row ids (rids). Here, this would correspond to only looking at certain genes of interest. 

<a id="annotations"></a>
## Add/delete annotations to an existing GCToo instance 

### Adding annotations
Now we have our sig_id annotations, and also a GCToo instance of expression data. This would probably be more useful to us as one unified object, so let's annotate our `vorinostat_only_gctoo` with the id information relevant to it: 

In [42]:
# first, we need to subset all the metadata information from our larger metadata DataFrame
vorinostat_sig_id_info =  sig_info[sig_info["pert_iname"] == "vorinostat"]
# let's look at a couple rows! 
vorinostat_sig_id_info[0:5]

,sig_id,pert_id,pert_iname,pert_type,cell_id,pert_idose,pert_itime,distil_id
15923,LJP007_A375_24H:A03,BRD-K81418486,vorinostat,trt_cp,A375,10.0 um,24 h,LJP007_A375_24H_X1_B20:A03|LJP007_A375_24H_X2_...
16302,LJP007_A549_24H:A03,BRD-K81418486,vorinostat,trt_cp,A549,10.0 um,24 h,LJP007_A549_24H_X1_F1B19:A03|LJP007_A549_24H_X...
16681,LJP007_ASC.C_24H:A03,BRD-K81418486,vorinostat,trt_cp,ASC.C,10.0 um,24 h,LJP007_ASC.C_24H_X1_B21:A03|LJP007_ASC.C_24H_X...
17056,LJP007_ASC_24H:A03,BRD-K81418486,vorinostat,trt_cp,ASC,10.0 um,24 h,LJP007_ASC_24H_X1_B21:A03|LJP007_ASC_24H_X2.A2...
17432,LJP007_CD34_24H:A03,BRD-K81418486,vorinostat,trt_cp,CD34,10.0 um,24 h,LJP007_CD34_24H_X2_B21:A03|LJP007_CD34_24H_X3....


In [43]:
# to add this data as our col_metadata_df, 
# the sig_id column needs to be the index of the metadata DataFrame
# For details on why this is, please refer to GCTX requirements: https://clue.io/connectopedia/gctx_format
vorinostat_sig_id_info.set_index("sig_id", inplace=True)
# now the data frame is indexed by sig_ids consistent with those of the data_df:
vorinostat_sig_id_info.index 

Index([u'LJP007_A375_24H:A03', u'LJP007_A549_24H:A03', u'LJP007_ASC.C_24H:A03',
       u'LJP007_ASC_24H:A03', u'LJP007_CD34_24H:A03', u'LJP007_HA1E_24H:A03',
       u'LJP007_HCC515_24H:A03', u'LJP007_HEPG2_24H:A03',
       u'LJP007_HME1_24H:A03', u'LJP007_HT29_24H:A03',
       ...
       u'LPROT003_A375_6H:P12', u'LPROT003_A549_6H:O08',
       u'LPROT003_A549_6H:O10', u'LPROT003_A549_6H:O12',
       u'LPROT003_NPC_6H:P07', u'LPROT003_NPC_6H:P09', u'LPROT003_NPC_6H:P11',
       u'LPROT003_PC3_6H:O07', u'LPROT003_PC3_6H:O09', u'LPROT003_PC3_6H:O11'],
      dtype='object', name=u'sig_id', length=210)

In [44]:
# set the relevant annotations as a col_metadata_df: 
vorinostat_only_gctoo.col_metadata_df = vorinostat_sig_id_info

### Deleting Annotations 

Hm, but now that we look at it we probably don't need the "pert_type" field; since we're only looking at vorinostat data, it's "trt_cp" (compound) for all samples. We could keep it anyway, but just for fun let's delete it: 

In [45]:
del vorinostat_only_gctoo.col_metadata_df["pert_type"]
vorinostat_only_gctoo.col_metadata_df[0:5]

,pert_id,pert_iname,cell_id,pert_idose,pert_itime,distil_id
cid,,,,,,
LJP007_ASC_24H:A03,BRD-K81418486,vorinostat,ASC,10.0 um,24 h,LJP007_ASC_24H_X1_B21:A03|LJP007_ASC_24H_X2.A2...
LJP007_HUES3_24H:A03,BRD-K81418486,vorinostat,HUES3,10.0 um,24 h,LJP007_HUES3_24H_X2_B24:A03|LJP007_HUES3_24H_X...
LJP007_MNEU.E_24H:A03,BRD-K81418486,vorinostat,MNEU.E,10.0 um,24 h,LJP007_MNEU.E_24H_X1_B24:A03|LJP007_MNEU.E_24H...
LJP007_NEU_24H:A03,BRD-K81418486,vorinostat,NEU,10.0 um,24 h,LJP007_NEU_24H_X1_B21:A03|LJP007_NEU_24H_X2_B2...
LJP007_NPC.CAS9_24H:A03,BRD-K81418486,vorinostat,NPC.CAS9,10.0 um,24 h,LJP007_NPC.CAS9_24H_X1_B21:A03|LJP007_NPC.CAS9...


<a id="slicing"></a> 
## Slicing GCToo instances 

Let's say that we're actually only interested in looking at vorinostat's signatures in A375 cells. First, are there vorinostat samples treated in this cell line in this GCToo instance? Let's check by first listing all unique cell lines present: 

In [46]:
vorinostat_only_gctoo.col_metadata_df["cell_id"].unique()

array(['ASC', 'HUES3', 'MNEU.E', 'NEU', 'NPC.CAS9', 'NPC', 'SKL', 'A375',
       'A549', 'PC3', 'ASC.C', 'CD34', 'HA1E', 'HCC515', 'HEPG2', 'HME1',
       'HT29', 'HUVEC', 'JURKAT', 'MCF7', 'NPC.TAK', 'SKL.C'], dtype=object)

Let's slice our GCToo instance to only include samples/columns corresponding to vorinostat-treated A375 cells.

In [47]:
from cmapPy.pandasGEXpress import slice_gct 

# first, get the cid values we want to keep 
vorinostat_a375_ids = vorinostat_only_gctoo.col_metadata_df.index[vorinostat_only_gctoo.col_metadata_df["cell_id"] == "A375"]

# slice GCToo instance to only include these values 
vorinostat_a375_gctoo = slice_gct.slice_gctoo(vorinostat_only_gctoo, cid=vorinostat_a375_ids)

# Now, our samples only correspond to one cell type: A375
vorinostat_a375_gctoo.col_metadata_df["cell_id"].unique()

array(['A375'], dtype=object)

Note that you can also slice by excluding rids/cids. For instance, if we wanted to include all cell lines **except** for A375, we could do so as follows: 

In [48]:
# slice vorinostat_only_gctoo to exclude A375 samples only
vorinostat_except_a375_gctoo = slice_gct.slice_gctoo(vorinostat_only_gctoo, exclude_cid=vorinostat_a375_ids)

# Now we have all cell lines except for A375: 
vorinostat_except_a375_gctoo.col_metadata_df["cell_id"].unique()

array(['ASC', 'HUES3', 'MNEU.E', 'NEU', 'NPC.CAS9', 'NPC', 'SKL', 'A549',
       'PC3', 'ASC.C', 'CD34', 'HA1E', 'HCC515', 'HEPG2', 'HME1', 'HT29',
       'HUVEC', 'JURKAT', 'MCF7', 'NPC.TAK', 'SKL.C'], dtype=object)

You could slice genes/rows of the GCToo analogously. 

<a link="merging"></a>
## Merging GCToo objects/.gct/.gctx files 

If you'd like to concatenate two or more objects/files, you have two options: 

1. Concatenate files from the command line 
2. Concatenate GCToo objects within a Python session. 

To continue with our GEO-based example, here we'll demonstrate option 2; for details on how to concatenate on the command line, please see [our documentation](https://clue.io/cmapPy/pandasGEXpress.html).

### Concatenating GCToo objects in a Python session 

We have our vorinostat expression data in A375; let's say we want to also get belinostat expression data in A375 (separately, for purposes of demonstration) and then combine these two GCToos. First, some prep: let's get the belinostat expression data in A375 that we need to do this operation: 

In [49]:
# get sig info corresponding to belinostat-treated A375 samples
belinostat_a375_sig_info =  sig_info[(sig_info["pert_iname"] == "belinostat") & (sig_info["cell_id"] == "A375")]
# set sig_id as index 
belinostat_a375_sig_info.set_index("sig_id", inplace=True)
# parse in only expression data corresponding to belinostat-treated A375 samples
belinostat_a375_gctoo = parse("GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328_2017-03-06.gctx", cid=belinostat_a375_sig_info.index)
# annotate belinostat_a375_gctoo with its corresponding sample annotations
belinostat_a375_gctoo.col_metadata_df = belinostat_a375_sig_info.copy()

And now we can concatenate our belinostat & vorinostat GCToos as follows:

In [50]:
from cmapPy.pandasGEXpress import concat_gctoo
# horizontally stack the two GCToo instances (note, you can also vertically stack)
# Somewhat confusingly, hstack concatenates samples together (same genes) and vstack does the opposite.
HDAC_a375_gctoo = concat_gctoo.hstack([vorinostat_a375_gctoo, belinostat_a375_gctoo])

# Now we have a gctoo corresponding to two compounds, in one cell line: 
print("unique compound data present: ") 
print(HDAC_a375_gctoo.col_metadata_df["pert_iname"].unique())
print("unique cell line data present: ")
print(HDAC_a375_gctoo.col_metadata_df["cell_id"].unique())

# Also note we still have the same genes as before, just with additional samples of data: 
print("combined gctoo expression data shape:")
print(HDAC_a375_gctoo.data_df.shape)

TypeError: hstack() takes at least 3 arguments (1 given)

### Concatenating two .gct/x files on the command line

We provide the command line tool `concat_gctoo` to concatenate either .gcts or .gctx files on the command line. 

Sample usage: 

` concat_gctoo --concat_direction horiz --input_filepaths first_file.gct second_file.gct` will output a file named concated.gctx; you can specify a different outname if you prefer. On the command line, type `concat_gctoo -h` to see help options. 

<a id="newGCToo"></a>
## Constructing a new GCToo instance programatically 

If you have your own DataFrames of data, you can also construct your own GCToo instance (note that to do so, you need to fulfill the [requirements of a valid gctx](clue.io/code)). Just for illustrative purposes, let's make one using copies of our HDAC GCToo DataFrames, so that we can also associate a single [multi-index DataFrame](http://pandas.pydata.org/pandas-docs/stable/advanced.html) of our combined data_df, row_metadata_df, and column_data_df with this GCToo instance. 

**Note**: By default, the GCToo constructor does not make a multi_index_df; details on this and other optional arguments [here](http://cmappy.readthedocs.io/en/latest/pandasGEXpress.html#gctoo-class).

In [ ]:
from cmapPy.pandasGEXpress import GCToo

HDAC_a375_gctoo_with_multiindex = GCToo.GCToo(data_df=HDAC_a375_gctoo.data_df.copy(), 
                                              row_metadata_df=HDAC_a375_gctoo.row_metadata_df.copy(), 
                                              col_metadata_df=HDAC_a375_gctoo.col_metadata_df.copy(), 
                                             make_multiindex=True)

# Let's take a look at a subset of this multiindex DataFrame; this can be useful for indexing etc.
HDAC_a375_gctoo_with_multiindex.multi_index_df[0:5]

For more information on multi-index DataFrames, please refer to the PANDAS project documentation. 

<a link="pandas"></a>
## Using pandas tools with your GCToo instance 

A big reason why we wrote `pandasGEXpress` was to enable users to maximize their use of `pandas`, `scikit-learn`, and other powerful and robustly-developed Python data science packages. Interacting with such tools simply requires referencing the relevant DataFrame attributes of your GCToo instance; here, we'll demonstrate how to do some basic arithmetic and grouping tasks on components of a GCToo instance. It should go without saying that the examples below are common but also fairly trivial examples of the wide functionality that libraries like pandas and sci-kit learn provide. 

Another `pandas` functionality that is likely to be of use is its `groupby` tools; as long as your data/annotations aren't too big, it can work nicely with your GCToo instance's `multi_index_df`.

### Column/row-wise means

Let's go back to our vorinostat-treated A375 cells. What is the mean expression across all samples? 

In [ ]:
import pandas as pd 

vorinostat_a375_gctoo.data_df.mean(axis = 0) #easy-peasy

If we wanted to determine the mean expression across all samples for individual genes, we would set axis=1. 

### Pairwise correlation of columns 

How correlated are our samples? 

In [ ]:
vorinostat_a375_cors = vorinostat_a375_gctoo.data_df.corr()
vorinostat_a375_cors[0:5] #only showing first 5 rows 

<a id="plotting"></a>
## Plotting with GCToo instances 

Because the attributes of a GCToo instance are pandas DataFrames, you can use any plotting libraries compatible with pandas DataFrames to generate plots; we frequently use [seaborn](https://seaborn.pydata.org/) and [matplotlib](https://matplotlib.org/). For simple plots, pandas also provides built-in plotting functionality for DataFrame objects.  

As an illustrative example, we calculate the correlations across samples again, but this time also plot it as a heatmap--all in one!

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()


sns.heatmap(vorinostat_a375_gctoo.data_df.corr(), cmap='plasma')
plt.show()

<a id="writing"></a>
## Writing GCToo instances to file 

Here's how you would write your GCToo instance as a .gctx; note that if you'd like to write to .gct, the syntax is identical but you need to import ``write_gct`` instead. 

In [ ]:
from cmapPy.pandasGEXpress import write_gctx as write_gctx

# write GCToo instance to .gctx
write_gctx.write(belinostat_a375_gctoo, "belinostat_a375")

<a id="converting"></a>
## Converting .gct <-> .gctx 

Converting from a gct to a gctx might be useful if you have a large gct and want faster IO in the future. We provide the following command line tools to do so: 

* `gct2gctx`
* `gctx2gct`

**To write some_thing.gct -> some_thing.gctx:**

gct2gctx -filename path/to/in_name.gct -output_filepath /path/to/out_name

Converting a gctx to a gct might be useful if you want to look at your .gctx file in a text editor or something similar. 

**To write some_thing.gctx -> some_thing.gct in working directory:**

gctx2gct -filename path/to/in_name.gctx -output_filepath /path/to/out_name
